In [19]:
import pandas as pd
import pyodbc
from tqdm.notebook import tqdm



  2%|▏         | 67043/3128626 [01:32<1:59:31, 426.92it/s]

In [4]:
username = 'vigrose'
password = 'Slytherin~10946'
q = "DSN=eprdods; UID={}; PWD={}".format(username, password)
ODS = pyodbc.connect(q)

In [5]:
#providers
iqvia_query = \
        """
        SELECT
        P.ME,
        A.AFFIL_IND,
        A.AFFIL_RANK,
        A.AFFIL_TYPE_ID,
        A.IMS_ORG_ID AS AFFIL,
        B.IMS_ORG_ID AS BEST_AFFIL
        FROM 
        ODS.ODS_IMS_PROFESSIONAL P, ODS.SAS_ODS_IMS_PROVIDER_AFFIL A, ODS.SAS_ODS_IMS_PROVIDER_BEST_AFFIL B
        WHERE  
        A.PROFESSIONAL_ID = P.PROFESSIONAL_ID
        AND
        P.CURRENT_BATCH_FLAG='Y'
        AND
        A.CURRENT_BATCH_FLAG='Y'
        AND
        P.ME IS NOT NULL
        AND
        B.PROFESSIONAL_ID = A.PROFESSIONAL_ID
        """
test = pd.read_sql(con=ODS, sql=iqvia_query)
test.head()

,ME,AFFIL_IND,AFFIL_RANK,AFFIL_TYPE_ID,AFFIL,BEST_AFFIL
0,7371002001,P,9,3.0,INS00000839,INS00824338
1,0200289093,P,10,NaN,INS00134683,INS00134683
2,6540198087,A,7,3.0,INS00261776,INS00454825
3,0500211012,A,8,1.0,INS00489748,INS00001621
4,7591816014,P,9,NaN,INS00478156,INS00478156


In [9]:
test_dict = test.to_dict('records')

In [20]:
predicted_best = {}
mes = []
for row in tqdm(test_dict):
    me = row['ME']
    if me not in mes:
        mes.append(me)
        predicted_best[me] = {}
        predicted_best[me]['RANK'] = row['AFFIL_RANK']
        predicted_best[me]['ORG'] = row['AFFIL']
    elif predicted_best[me]['RANK'] < row['AFFIL_RANK']:
        predicted_best[me]['RANK'] = row['AFFIL_RANK']
        predicted_best[me]['ORG'] = row['AFFIL']
    elif predicted_best[me]['RANK'] == row['AFFIL_RANK']:
        if row['AFFIL_IND'] == 'P':
            predicted_best[me]['RANK'] = row['AFFIL_RANK']
            predicted_best[me]['ORG'] = row['AFFIL']

KeyboardInterrupt: 

In [21]:
test = test.sort_values(['AFFIL_RANK','AFFIL_IND'])

In [24]:
predicted_best = test.drop_duplicates('ME', keep='last')

In [26]:
predicted_best[predicted_best.AFFIL!=predicted_best.BEST_AFFIL]

,ME,AFFIL_IND,AFFIL_RANK,AFFIL_TYPE_ID,AFFIL,BEST_AFFIL
59542,0350193035,A,6,3.0,INS00005104,INS00005087
60677,0161174045,A,6,3.0,INS00005016,INS00532166
69334,0172063117,A,6,3.0,INS00103618,INS00003223
71387,0260877017,A,6,3.0,INS00004876,INS00005619
73785,0480475228,A,6,2.0,INS00005449,INS00536183
...,...,...,...,...,...,...
3128320,0384092246,P,10,NaN,INS01040800,INS01062878
3128555,0481203106,P,10,3.0,INS00003971,INS01071066
3128584,0164377056,P,10,1.0,INS00005196,INS00909223
3128592,0240786150,P,10,NaN,INS00883783,INS00745545


In [ ]:
#how often is best not preferred
#is best ever not top ranked